# Import libraries and tools

In [7]:
import os
import numpy as np
import pandas as pd
from PIL import Image, ImageEnhance
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications import VGG16
from sklearn.utils import shuffle


# Load Dataset

In [13]:
train_dir = "MRI_Images/training"
test_dir = "MRI_Images/testing"

train_paths = []
train_labels = []

for label in os.listdir(train_dir):
    for image in os.listdir(os.path.join(train_dir, label)):
       train_paths.append(os.path.join(train_dir, label, image))
       train_labels.append(label)

train_paths, train_labels = shuffle(train_paths, train_labels)


test_paths = []
test_labels = []

for label in os.listdir(test_dir):
    for image in os.listdir(os.path.join(test_dir, label)):
       test_paths.append(os.path.join(test_dir, label, image))
       test_labels.append(label)

test_paths, test_labels = shuffle(test_paths, test_labels)

# Data Visualization